In [1]:
# khai báo thư viện
import tensorflow
import tensorflow as tf
import cv2
import numpy
import matplotlib.pyplot as plt
import glob
import PIL
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model
from keras.models import Sequential,Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
classifier = Sequential()
model = Sequential()

Using TensorFlow backend.


In [0]:
import functools
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1)
top3_acc.__name__ = 'top3_acc'
top1_acc.__name__ = 'top1_acc'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#đọc đường dẫn của tất cả các ảnh
link =[]
label=[]
i=0
dog_name = ["bullmastiff","chowchow","pug","maltese","huskysibir","dachshund","dalmatian","corgi","chihuahua","yorkshire"]
for name in dog_name:
    for file in glob.glob("/content/drive/My Drive/dataset/" + name + "/*JPEG"):
      link.append(file)
      label.append(i)
    i+=1

In [0]:
#def generator
def data_gen(link,label,size=10):
  while True:
    data=[]
    lab=[]
    index = numpy.random.choice(len(link),10)
    for i in range(10):
      img = cv2.imread(link[index[i]])
      img = cv2.resize(img,(224,224))
      data.append(img)
      lab.append(label[index[i]])
    batch_data = numpy.asarray(data, dtype=numpy.float32)
    batch_label=tf.keras.utils.to_categorical(lab,num_classes=10)
    yield (batch_data,batch_label)
  

In [0]:
#chia link thành train_link và validation_link
split = int(0.8*len(link))
train_link=link[0:split]
valid_link=link[split:]
train_label=label[0:split]
valid_label=label[split:]

In [0]:
#mô hình
vgg = keras.applications.vgg16.VGG16(include_top=True,weights="imagenet")
for layer in vgg.layers:
  layer.trainable = False

softmax = Dense(10,activation='softmax')(vgg.layers[-2].output)

model = Model(vgg.input,softmax)
model.summary()

In [0]:
train_step = len(train_link) // 10
valid_step = len(valid_link) // 10

In [0]:
train_gen = data_gen(train_link,train_label,size=10)
valid_gen = data_gen(valid_link,valid_label,size=10)

In [35]:
print("[INFO] training network...")
opt = Adam(lr=1e-3, decay=1e-3 / 50)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=['top_k_categorical_accuracy',top1_acc,top3_acc])
H = model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=train_step,validation_steps = valid_step,verbose=1,use_multiprocessing=True,epochs=1)

[INFO] training network...
Epoch 1/1
902/902 [==============================] - 12075s 13s/step - loss: 0.4237 - top_k_categorical_accuracy: 0.9963 - top1_acc: 0.9190 - top3_acc: 0.9866 - val_loss: 1.5947 - val_top_k_categorical_accuracy: 0.9498 - val_top1_acc: 0.8329 - val_top3_acc: 0.9138
